##### **Fetch card images**

In [ ]:
from art_scan import process_query

queries = [
    "Balefire Dragon",
]

[process_query(q) for q in queries];

##### **Alter File MD5**

In [ ]:
from typing import Tuple
import os


def parse_file_path(path: str, file_separator: str = "/") -> Tuple[str, str, str]:
    """Parse a file path into the full directory, file name, and extension. 

    Args:
        path (str): Full path to file
        file_separator (str, optional): File separator to use. Defaults to "\\".

    Returns:
        tuple[str, str, str]: A tuple with three elements containing:
            - Full directory path to the file
            - Name of the file (without full directory path)
            - Extension of the file
    """
    path_tokens = path.split(file_separator)
    file_name = path_tokens[-1]
    file_path = file_separator.join(path_tokens[:-1])
    
    ext_tokens = file_name.split('.')
    file_name = ext_tokens[0] if len(ext_tokens) <= 1 else '.'.join(ext_tokens[:-1])
    file_ext = ext_tokens[-1]

    return (file_path, file_name, file_ext)


def duplicate_file(source_path: str, destination_path: str, count: int):
    file_contents = None
    with open(source_path, 'rb') as orig_file:
        file_contents = orig_file.read()

    _, file_name, file_ext = parse_file_path(source_path)
    for i in range(count):
        alt_file_path = os.path.join(destination_path, f"{file_name} [{i + 1}].{file_ext}")
        with open(alt_file_path, 'wb') as alt_file:
            alt_file.write(file_contents + (i * b'\0'))

# image_path = "C:\\Users\\evanh\\My Drive\\MTG\\Proxies\\mpc-ready\\Artifact\\Aetherflux Reservoir (Extended).png"
# destination_path = "C:\\Users\\evanh\\My Drive\\MTG\\Proxies\\temp"
# duplicate_file(image_path, destination_path, 2)

# CHECK
# hashlib.md5(open(image_path, 'rb').read()).hexdigest()

##### **Pull Private Proxy List from Google Sheets**

In [ ]:
# from collections import namedtuple
# from googleapiclient import discovery
# from lib.google_auth import get_creds

# INV_SHEETS_ID = '13ofTLZFmPMkT09Np2US8KF_JmTfgqdeQ5aeY16L43s0'
# INV_RANGE = 'Inventory!A:L'

# service = discovery.build('sheets', 'v4', credentials=get_creds())

# # Call the Sheets API
# sheets_args = dict(
#     spreadsheetId=INV_SHEETS_ID,
#     range=INV_RANGE,
# )
# result = service.spreadsheets().values().get(**sheets_args).execute()
# proxy_list = list(result.get('values', []))

##### **Pull Public Proxy List from Google Sheets**

In [ ]:
import requests
import csv

sheet_id = '13ofTLZFmPMkT09Np2US8KF_JmTfgqdeQ5aeY16L43s0'
url = f'https://docs.google.com/spreadsheet/ccc?key={sheet_id}&output=csv'

# sheet_id = "1QRqqMGxbZWZYaa9UXeE3PwrQxZjRh7i_3TNTzqwlN5Y"
# sheet_name = "Inventory"
# url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

with requests.Session() as s:
    download = s.get(url)
    decoded_content = download.content.decode('utf-8')
    proxy_list = list(csv.reader(decoded_content.splitlines(), delimiter=','))


##### **Load List in Python Objects**

In [ ]:
from dataclasses import dataclass
# from datetime import datetime

@dataclass
class Proxy:
    """Class to store relevant proxy information."""
    name: str
    type: str
    # last_edited: datetime
    order_count: int

    COL_MAP = {
        "name": 0,
        "type": 1,
        # "last_edited": 2,
        "order_count": 11,
    }
    DATE_FORMAT_STR = "%b %d %H:%M"
    
    @property
    def is_mdfc(self) -> bool:
        return '//' in self.name

    @classmethod
    def from_row(cls, row):
        return cls(
            name=row[cls.COL_MAP['name']],
            type=row[cls.COL_MAP['type']],
            # last_edited=datetime.strptime(row[cls.COL_MAP['last_edited']], cls.DATE_FORMAT_STR),
            order_count=int(row[cls.COL_MAP['order_count']])
        )

# Load non-empty rows into `Proxy` type
all_proxies = [Proxy.from_row(p) for p in proxy_list[4:] if len(p[0]) > 0]
proxies_to_print = [p for p in all_proxies if p.order_count > 0]
len(proxies_to_print)

##### **Find missing proxy files**

In [ ]:
import os

PROXY_DIR = "C:/Users/evanh/My Drive/MTG/Proxies/mpc-ready"

card_types = list(set([p.type for p in proxies_to_print]))
proxies_by_type = {
    type_: [p for p in proxies_to_print if p.type == type_]
    for type_ in card_types
}

for card_type in card_types:
    files = os.listdir(f"{PROXY_DIR}/{card_type}")
    for proxy in proxies_by_type[card_type]:
        found = False
        for file in files:
            if proxy.name in file:
                found = True
                break
        if not found:
            print(proxy)

##### **Write out files for order**

In [ ]:
FOLDER_SIZE = 55
FOLDER_NAME = "proxies_{number:02d}"
TEMP_FOLDER = "C:/Users/evanh/Temp/landing"
OUTPUT_FOLDER = "C:/Users/evanh/Temp/mpc"
PROXY_DIR = "C:/Users/evanh/My Drive/MTG/Proxies/mpc-ready"

# Write all files out to landing zone
for proxy in proxies_to_print:
    search_path = f"{PROXY_DIR}/{proxy.type}"
    file_names = [f for f in os.listdir(search_path) if f.startswith(f"{proxy.name}.")]

    if len(file_names) < 1:
        print(f"Unable to find [{proxy.name}] at [{search_path}]")
        continue
    elif len(file_names) > 1:
        print(f"Found multiple matches for {proxy.name} at [{search_path}]")
        print(file_names)
        continue

    proxy_path = f"{PROXY_DIR}/{proxy.type}/{file_names[0]}"
    duplicate_file(proxy_path, TEMP_FOLDER, proxy.order_count)

# Move files into subfolders of fixed size (`FOLDER_SIZE`)
current_folder = 0
while len(os.listdir(TEMP_FOLDER)) > 0:
    # Create temp folder
    output_folder = os.path.join(OUTPUT_FOLDER, FOLDER_NAME.format(number=current_folder))
    os.mkdir(output_folder)

    # Move a number of files up to the configured folder size
    for file in os.listdir(TEMP_FOLDER)[:FOLDER_SIZE]:
        source_file = os.path.join(TEMP_FOLDER, file)
        output_file = os.path.join(output_folder, file)
        os.rename(source_file, output_file)

    current_folder += 1

##### **Handle MDFCs separately**

In [ ]:
mdfc_proxies = [p for p in proxies_to_print if p.is_mdfc]
for proxy in mdfc_proxies:
    for name in proxy.name.split(' // '):
        search_path = f"{PROXY_DIR}/{proxy.type}"
        file_names = [f for f in os.listdir(search_path) if f.startswith(f"{name}.")]

        if len(file_names) < 1:
            print(f"Unable to find [{name}] at [{search_path}]")
            continue
        elif len(file_names) > 1:
            print(f"Found multiple matches for {name} at [{search_path}]")
            print(file_names)
            continue

        proxy_path = f"{PROXY_DIR}/{proxy.type}/{file_names[0]}"
        duplicate_file(proxy_path, TEMP_FOLDER, proxy.order_count)
        